In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from datetime import timedelta

import geopandas
import geopy
from geopy.geocoders import Nominatim
import folium
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import MarkerCluster

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_csv('D:/darsh/GeoCrimeMapping/addresses.csv')
df.head()c

,case,crime,occurred,comments,building,address,disposition
0,1905347,SUSPICIOUS EVENT,04-02-2023 21:39,~,MYSORE PALACE,"Chamrajpura, Mysuru, Karnataka 570001",Inactive
1,1905348,DRUG INCIDENT,05-02-2023 10:39,~,Sri Ganapathy Sachchidananda Ashrama,"JP Nagara, Mysuru, Karnataka 570025",Unfounded
2,1905349,TRAFFIC STOP,06-02-2023 01:39,~,Sizzling Corner,"Kuvempu Nagara, Mysuru, Karnataka 570023",Clear by Warning
3,1905350,DRUG INCIDENT,07-02-2023 02:39,~,Study Centre / Reading Room - IAS kannadiga,"Krishnamurthy Puram, Mysuru, Karnataka 570004",Closed
4,1905351,ASSIST EMS POLICE,08-02-2023 13:39,~,"Government Hospital, TK layout","TK Layout, Mysuru, Karnataka 570009",Inactive


In [3]:
df.dropna(subset=['occurred'], inplace=True)

df['occurred'] = pd.to_datetime(df['occurred'])
df['date'] = [d.date() for d in df['occurred']]
df['time'] = [d.time() for d in df['occurred']]
df['day'] = df['occurred'].dt.day_name()
df['timeint'] = (df['occurred']-df['occurred'].dt.normalize()).dt.total_seconds()/timedelta(days=1).total_seconds()
df = df[df['disposition'] != 'Unfounded']
df.head()

,case,crime,occurred,comments,building,address,disposition,date,time,day,timeint
0,1905347,SUSPICIOUS EVENT,2023-04-02 21:39:00,~,MYSORE PALACE,"Chamrajpura, Mysuru, Karnataka 570001",Inactive,2023-04-02,21:39:00,Sunday,0.902083
2,1905349,TRAFFIC STOP,2023-06-02 01:39:00,~,Sizzling Corner,"Kuvempu Nagara, Mysuru, Karnataka 570023",Clear by Warning,2023-06-02,01:39:00,Friday,0.068750
3,1905350,DRUG INCIDENT,2023-07-02 02:39:00,~,Study Centre / Reading Room - IAS kannadiga,"Krishnamurthy Puram, Mysuru, Karnataka 570004",Closed,2023-07-02,02:39:00,Sunday,0.110417
4,1905351,ASSIST EMS POLICE,2023-08-02 13:39:00,~,"Government Hospital, TK layout","TK Layout, Mysuru, Karnataka 570009",Inactive,2023-08-02,13:39:00,Wednesday,0.568750
5,1905352,VADALISM,2023-09-02 20:39:00,~,University Of Mysore Campus,"University Of Mysore Campus, Mysuru, Karnataka",Inactive,2023-09-02,20:39:00,Saturday,0.860417


In [4]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Mysore, Karnataka')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

df = df[df['point'].notnull()]
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

,case,crime,occurred,comments,building,address,disposition,date,time,day,timeint,location,point,latitude,longitude,altitude
2,1905349,TRAFFIC STOP,2023-06-02 01:39:00,~,Sizzling Corner,"Kuvempu Nagara, Mysuru, Karnataka 570023",Clear by Warning,2023-06-02,01:39:00,Friday,0.068750,"(Kuvempu Statue, Kuvempunagara, Mysuru, Mysuru...","(12.2927606, 76.625013, 0.0)",12.292761,76.625013,0.0
4,1905351,ASSIST EMS POLICE,2023-08-02 13:39:00,~,"Government Hospital, TK layout","TK Layout, Mysuru, Karnataka 570009",Inactive,2023-08-02,13:39:00,Wednesday,0.568750,"(TK Layout Park, Ramakrishna Nagara, Mysuru, M...","(12.2972892, 76.61933754500285, 0.0)",12.297289,76.619338,0.0
5,1905352,VADALISM,2023-09-02 20:39:00,~,University Of Mysore Campus,"University Of Mysore Campus, Mysuru, Karnataka",Inactive,2023-09-02,20:39:00,Saturday,0.860417,"(University School of Design, Open Air Theater...","(12.3136181, 76.6248404, 0.0)",12.313618,76.624840,0.0
6,1905353,ASSIST FIRE DEPT POLICE,2023-10-02 15:39:00,~,RAILWAY MUSEUM MYSURU,"Yadavagiri, Mysuru, Karnataka 570005",Closed,2023-10-02,15:39:00,Monday,0.652083,"(Yadavagiri, Mysuru, Mysuru taluk, Mysuru dist...","(12.327919, 76.6391525, 0.0)",12.327919,76.639152,0.0
9,1905356,DACOITY,2023-12-03 11:39:00,~,SUYOG HOSPITAL,"Ramakrishnanagar, Mysuru, Karnataka 570023",Inactive,2023-12-03,11:39:00,Sunday,0.485417,"(Gaanu Chiru Medicals, #2, Tibbadevi Circle, R...","(12.2927168, 76.6166856, 0.0)",12.292717,76.616686,0.0


In [5]:
df.groupby("crime")["crime"].count().sort_values()

crime
ASSIST EMS POLICE          20
ASSIST FIRE DEPT POLICE    20
DACOITY                    20
HURT                       20
KIDNAPPING                 20
LOUD MUSIC                 20
MURDER                     20
RIOTS                      20
VADALISM                   20
TRAFFIC STOP               40
Name: crime, dtype: int64

In [6]:
df.groupby("building")["building"].count().sort_values()

building
Amrita Vishwa Vidyapeetham, Mysore Campus    20
BEML Administrative Building                 20
Government Hospital, TK layout               20
Grand Mercure Mysore                         20
Lingambudhi Park                             20
RAILWAY MUSEUM MYSURU                        20
SUYOG HOSPITAL                               20
Sharmada Hidden Nest Resort                  20
Sizzling Corner                              20
Sri Chamundeshwari Temple                    20
University Of Mysore Campus                  20
Name: building, dtype: int64

In [7]:
df.groupby("date")["date"].count().sort_values()

date
2023-02-03    20
2023-05-03    20
2023-06-02    20
2023-08-02    20
2023-09-02    20
2023-10-02    20
2023-12-03    20
2023-12-05    20
2023-12-02    60
Name: date, dtype: int64

In [8]:
m = folium.Map([39.645,-79.96], zoom_start=2)
for index, row in df.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=3,
                        popup=row['crime'],
                        fill_color="#3db7e4",
                       ).add_to(m)

In [9]:
dfmatrix = df[['latitude', 'longitude']].values
m.add_child(plugins.HeatMap(dfmatrix, radius=15))
m